In [ ]:
#get_file_word.py
#!/usr/bin/env python
# coding: utf-8

In [ ]:
# metodo para ler arquivo word

In [1]:
# importar biblioteca
import os
# montagem de diretório na rede a partir de um diretório local (pastalocal)
#mkdir pastaLocal
dir_destino= "pastaLocal"
if not os.path.exists(dir_destino):
    os.makedirs(dir_destino)

In [2]:
print("O diretório "+dir_destino+" exite?",os.path.exists(dir_destino))

O diretório pastaLocal exite? True


In [59]:
# achando o caminho dentro da rede - MONTANDO DIRETÓRIO NA REDE
!mount_smbfs smb://ensi-filer02/gerpubprop$ pastaLocal/
##### se der ERRO nesse comando ejetar a pasta gerpubprop$ antes e rodar o codigo novamente

In [60]:
#TESTANDO O MONTAGEM DO DIRETÓRIO
os.listdir('/Users/wpessoa/repositorios/portifolio_nge/pastaLocal/Coordenacao de Gestao Editorial/2021/TRIAGEM/')

['FEITO',
 '.smbdeleteAAA1c13d76d004af55b',
 '~$pre-triagem-2021.xlsx',
 'ID236690-Novas barreiras e tendências no comércio internacional.docx',
 'formato_data.xlsx']

In [65]:
import get_file_word as gf
import datetime
import pandas as pd
from datetime import date
import shutil
import subprocess
import re
import docx

# capturar data do sistema
data_atual = date.today()

#setar variáveis onde estão arquivos da triagem e tipo do arquivo word (docx)
fileExt = r".docx"
dir_origem = r"/Users/wpessoa/repositorios/portifolio_nge/pastaLocal/Coordenacao de Gestao Editorial/2021/TRIAGEM/"

# verificar se existe o diretório
print("Existe esse diretório? ",os.path.exists(dir_origem)) 

#criar variável contendo todos os arquivos com extensão DOCX no diretório especifico do dir_origem 
files_array = [_ for _ in os.listdir(dir_origem) if _.endswith(fileExt)]

print("Quantidade de arquivos do word no diretório: ",len(files_array))
# print(files_array)
print("-------------------------------------------------------------")

# rodar método get_info para ler aquivos do Word e retornar features de cada arquivo
# argumentos (diretório e array dos arquivos word)
# retorna (NºID,Qtd_caracteres,Qtd_tabela,Qtd_image,data) e gera variável triagem_docx

triagem_docx=gf.get_info(dir_origem,files_array)


# gerar um Dataframe a partir do método get_file_word (gf)
df_triagem_docx = pd.DataFrame(triagem_docx,columns=['Nun_ID',"Qtd_PG_word","Qtd_carac","Qtd_tabela","Qtd_image","Qtd_estilos", "Data"])

# inserindo data da triagem
df_triagem_docx['Dt_triagem'] = data_atual
df_triagem_docx["Apresentação"]= None


# OUTPUT das variáveis do método
for i in range(0,len(df_triagem_docx.Nun_ID)):
    #print("i= ",i)
    print(files_array[i])
    print("Número de ID: ",df_triagem_docx.Nun_ID[i])
    print("Qtd de páginas: ",df_triagem_docx.Qtd_PG_word[i])
    #print("Qtd de caracteres: ",df_triagem_docx.Qtd_carac[i])
    #print("Qtd de tabelas: ",df_triagem_docx.Qtd_tabela[i])
    #print("Qtd de imagens: ",df_triagem_docx.Qtd_image[i])
    #print("Qtd de estilos no Word: ",df_triagem_docx.Qtd_estilos[i])
    #print("Data de criação do documento: ",df_triagem_docx.Data[i])
    #df_triagem_docx.loc[[i]].to_excel("/Users/wpessoa/repositorios/portifolio_nge/pastaLocal/Coordenacao de Gestao Editorial/2021/TRIAGEM/pre-triagem-ID"+df_triagem_docx.Nun_ID[i]+".xlsx",index=False,header=True )
    
    
    #for file in files_array:
    # metodo para veriricar se o texto word tem Apresentação
    try: 
        #print(files_array[i])
        doc = docx.Document(dir_origem+files_array[i])
        # ler em cada parágrafo dentro do arquivo Word
        paragra= [p.text for p in doc.paragraphs]
        for paragrafo in list(paragra):
            # str_extract_all(text_1, regex(pattern = 'f.*',ignore_case = TRUE, multiline = FALSE))
            print(paragrafo)
            if (re.search('^Apresentação$', paragrafo, re.IGNORECASE))or(re.search('^Apresentação $', paragrafo, re.IGNORECASE)):
                print("APRESENTAÇÃO ENCONTRADA")
                #print(paragrafo)
                df_triagem_docx.loc[i, 'Apresentação']="sim"
            if (re.search('\f', paragrafo)):
                print(paragrafo)
    except:       
        print("erro leitura de arquivo")
    print("-------------------------------------------------------------")

# recuperar conteudo do arquivo Excel
excelFile_old = dir_origem+"pre-triagem"+"-"+str(data_atual.year)+".xlsx"

if os.path.exists(excelFile_old):
    #print("EXISTE ARQUIVO EXCEL")
    excelFile = pd.read_excel(excelFile_old)    
    #df_triagem_docx.append(excelFile)
    # adicionar dataframe do arquivo excel anterior no dataframe da saida do método  get_file_word
    
    excelFile = excelFile.append(df_triagem_docx)
    
    # --------------------------------
    # alterar o formato da data
    excelFile['Dt_triagem'] = pd.to_datetime(excelFile.Dt_triagem)
    excelFile['Dt_triagem'] = excelFile['Dt_triagem'].dt.strftime('%d/%m/%Y')

    # --------------------------------
    # alterar o formato da data
    excelFile['Data'] = pd.to_datetime(excelFile.Data)
    excelFile['Data'] = excelFile['Data'].dt.strftime('%d/%m/%Y')

    os.remove(excelFile_old)

    # --------------------------------
    excelFile.to_excel(dir_origem+"pre-triagem"+"-"+str(data_atual.year)+".xlsx",index=False,header=True )
    
else:
    #print("(((  NÃO  )))  EXISTE ARQUIVO EXCEL")
    # gerar arquivo excel para arquivo tratado pela primeira vez
    df_triagem_docx.to_excel(dir_origem+"pre-triagem"+"-"+str(data_atual.year)+".xlsx",index=False,header=True )

# copiara os arquivo word para pasta de processo feito
dir_destino = dir_origem+"/FEITO/"
if len(files_array)>0:
    try:
        for linha in files_array:
            if not os.path.exists(dir_destino):
                os.makedirs(dir_destino)
            shutil.move(dir_origem+ linha, dir_destino + linha)
    except OSError:
            print("Oops!  erro no método copiara arquivo - ERRO NO S.O.")

Existe esse diretório?  True
Quantidade de arquivos do word no diretório:  1
-------------------------------------------------------------
ID233885-1- Metodol Rec Did - rev.docx
Número de ID:  233885
Qtd de páginas:  72





MANUAL PARA DESENVOLVIMENTO DE RECURSOS DIDÁTICOS

Metodologia SENAI de Desenvolvimento de Recursos Didáticos


















Programa SENAI de Desenvolvimento de Recursos Didáticos
SENAI Departamento Nacional


CONFEDERAÇÃO NACIONAL DA INDÚSTRIA – CNI
Robson Braga de Andrade
Presidente

Gabinete da Presidência
Teodomiro Braga da Silva
Chefe do Gabinete - Diretor

Diretoria de Educação e Tecnologia - DIRET
Rafael Esmeraldo Lucchesi Ramacciotti
Diretor de Educação e Tecnologia

Serviço Social da Indústria - SESI
Eduardo Eugenio Gouvêa Vieira
Presidente do Conselho Nacional

SESI – Departamento Nacional
Robson Braga de Andrade
Diretor

Rafael Esmeraldo Lucchesi Ramacciotti
Diretor-Superintendente 

Paulo Mól Júnior
Diretor de Operações

Serviço Nacional de Aprendiz

In [57]:
#DESMONTANDO DIRETÓRIO NA REDE ====== precisa fechar todos arquivos que estiver usando a Pata que foi MONTADA
!umount pastaLocal/


In [52]:
#import datetime
#type(df_triagem_docx["Data"])
#df_triagem_docx["Data"].dt.date
#excelFile['Data'] = 
#print(pd.to_datetime(excelFile.Data))
#print(df_triagem_docx["Data"])

#df.date_time.map(lambda x: x.strftime('%Y-%m-%d'))
#df_triagem_docx.date_time.map(lambda x: x.strftime('%Y-%m-%d'))
#df_triagem_docx.dt.year

#data_em_texto = ‘{}/{}/{}’.format(data_atual.day, data_atual.month,data_atual.year)

#print("Ano: ",df_triagem_docx["Data"][0].year)

#data_atual.strftime(‘%d/%m/%Y’)

# df['Datetime'] = pd.to_datetime(df['Datetime'], format="%m/%d/%Y, %H:%M:%S")
#df_triagem_docx["Data"] = pd.to_datetime(df_triagem_docx['Data'], format="%m-%d-%Y")

#%timeit pd.to_datetime(df['Datetime'], format="%m/%d/%Y, %H:%M:%S")
#%timeit df['Datetime'].apply(lambda _: datetime.strptime(_,"%m/%d/%Y, %H:%M:%S"))
#df_triagem_docx["Data"] = df_triagem_docx['Data'].apply(lambda _:datetime.strptime(_,"%m/%d/%Y, %H:%M:%S"))
#df['DOB1'] = df['DOB'].dt.strftime('%m/%d/%Y')

#df_triagem_docx["Data"] = df_triagem_docx["Data"].dt.strftime('%m/%d/%Y')
print(type(df_triagem_docx["Data"][0]))
df_triagem_docx["Data"] = pd.to_datetime(df_triagem_docx["Data"])

#yields_df['Date'] = pd.to_datetime(yields_df['Date'], infer_datetime_format=True)
df_triagem_docx["Data"] = pd.to_datetime(df_triagem_docx["Data"], infer_datetime_format=True)

#df_triagem_docx["Data"] = pd.to_datetime(df_triagem_docx['Data'], format="%m/%d/%Y")


print(df_triagem_docx["Data"])
print(type(df_triagem_docx["Data"][0]))



<class 'pandas._libs.tslibs.timestamps.Timestamp'>
0   2020-02-05
1   2020-02-05
2   2019-12-06
3   2021-01-25
Name: Data, dtype: datetime64[ns]
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [142]:
df_triagem_docx

,Nun_ID,Qtd_PG_word,Qtd_carac,Qtd_tabela,Qtd_image,Qtd_estilos,Data,Dt_triagem,Apresentação
0,233885,72,97009,19,9,18,2020-02-05,2021-01-27,sim
1,233886,50,71279,37,32,15,2020-02-05,2021-01-27,sim
2,233867,31,58108,40,43,17,2019-12-06,2021-01-27,sim
3,236690,49,80889,46,55,26,2021-01-25,2021-01-27,sim


In [127]:
df_triagem_docx.loc[2, 'Apresentação']="sim"


In [ ]:
'''
Oops! XML não encontrado para file: ID234530-1- Metodol Rec Did-erro.docx
---------------------------------------------------------------------------
UnboundLocalError                         Traceback (most recent call last)
<ipython-input-6-9ba5c2b6e97c> in <module>
     26 # retorna (NºID,Qtd_caracteres,Qtd_tabela,Qtd_image,data) e gera variável triagem_docx
     27 
---> 28 triagem_docx=gf.get_info(dir_origem,files_array)
     29 
     30 

~/repositorios/portifolio_nge/get_file_word.py in get_info(fileDir, files_array)
     98 
     99                 # gerar lista resposta
--> 100                 list_retur=file[2:8],pagina_xml,sum(caracteres), len(table_chunks), len(image_array),len(array_styles), dt_doc.date()
    101 
    102                 # gerar tabela resposta

UnboundLocalError: local variable 'pagina_xml' referenced before assignment

In [95]:
!ls /Users/wpessoa/repositorios/portifolio_nge/pastaLocal/Coordenacaode Gestao Editorial/2021/TRIAGEM/

ls: /Users/wpessoa/repositorios/portifolio_nge/pastaLocal/Coordenacao: No such file or directory
ls: Editorial/2021/TRIAGEM/: No such file or directory
ls: Gestao: No such file or directory
ls: de: No such file or directory


In [97]:
import subprocess
import re
import docx

for file in files_array:
    try: 
        print(file)
        doc = docx.Document(dir_origem+file)
        # ler em cada parágrafo dentro do arquivo Word
        paragra= [p.text for p in doc.paragraphs]
        for paragrafo in list(paragra):
            if (re.search('^Apresentação$', paragrafo, re.IGNORECASE))or(re.search('^Apresentação $', paragrafo, re.IGNORECASE)):
                print("Apresentação encontrada")
                print(paragrafo)
    except:
        print("erro leitura de arquivo")

ID233867-06_12_2019_EFEITOS DA GESTÃO SOBRE A PRODUTIVIDADE - PARA PUBLICAÇÃO_Final_19122019_155125.docx
Apresentação encontrada
Apresentação 
ID233885-1- Metodol Rec Did - rev.docx
Apresentação encontrada
APRESENTAÇÃO
~$233867-06_12_2019_EFEITOS DA GESTÃO SOBRE A PRODUTIVIDADE - PARA PUBLICAÇÃO_Final_19122019_155125.docx
erro leitura de arquivo
~$233885-1- Metodol Rec Did - rev.docx
erro leitura de arquivo
ID233886-2- Metodol LD - rev.docx
Apresentação encontrada
APRESENTAÇÃO
ID236690-Novas barreiras e tendências no comércio internacional.docx
Apresentação encontrada
Apresentação
